# 🚀 Jetson Emotion Detection: Train + TensorRT Inference

In [1]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:

# === CONFIG ===
image_size = (96, 96)
batch_size = 32
epochs = 25
train_dir = "dataset/train"
test_dir = "dataset/test"


In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
class_labels = list(train_generator.class_indices.keys())


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(96, 96, 3)),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 94, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 47, 47, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 45, 45, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 22, 22, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 256)       0

In [2]:

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stop]
)


NameError: name 'EarlyStopping' is not defined

In [ ]:

# === SAVE FOR TensorRT ===
os.makedirs("saved_model", exist_ok=True)
model.save("saved_model")
print("✅ Model saved to 'saved_model/'")


In [ ]:

# === CONVERT TO TENSORRT ===
from tensorflow.python.compiler.tensorrt import trt_convert as trt

converter = trt.TrtGraphConverterV2(input_saved_model_dir="saved_model")
converter.convert()
converter.save("trt_model")
print("✅ TensorRT optimized model saved to 'trt_model/'")


In [ ]:

# === RUN TENSORRT INFERENCE ===
import tensorflow as tf

image_dir = "images"
infer_model = tf.saved_model.load("trt_model")
infer_fn = infer_model.signatures['serving_default']

for img_file in os.listdir(image_dir)[:5]:
    path = os.path.join(image_dir, img_file)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, image_size)
    norm = img.astype("float32") / 255.0
    norm = np.expand_dims(norm, axis=0)  # Shape: (1, 48, 48, 1)
    tensor_input = tf.convert_to_tensor(norm)

    result = infer_fn(tensor_input)
    preds = list(result.values())[0].numpy()[0]
    label = class_labels[np.argmax(preds)]
    conf = np.max(preds)

    print(f"{img_file} → {label} ({conf*100:.2f}%)")
    plt.imshow(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB))
    plt.title(f"{label} ({conf*100:.2f}%)")
    plt.axis("off")
    plt.show()
